In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
from mongoengine import connect, NotUniqueError
connect(
    db=os.getenv('MONGO_DB'),
    host=os.getenv('MONGO_URL'),
    port=27017,
    username=os.getenv('MONGO_USR'),
    password=os.getenv('MONGO_PWD'),
    authentication_source="admin"
    )

In [ ]:
from profile_model import ProfileModel, LocationHistoryModel
from mongoengine import connect, Document, LongField, StringField
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
import math

from stalkr import select_indexes, localizeProfile, visualize

In [ ]:
current_millis = int(time.time() * 1000)

# 1 hour ago in milliseconds
one_hour_ago_millis = current_millis - 0.2 * (60 * 60 * 1000)

# Aggregation pipeline to filter, group by batch_timestamp, and collect profileIds
pipeline = [
    {
        "$match": {
            "batch_timestamp": {"$gt": one_hour_ago_millis}
        }
    },
    {
        "$group": {
            "_id": "$batch_timestamp",  # Grouping by batch_timestamp
            "profileIds": {"$addToSet": "$profileId"}  # Collecting unique profileIds for each batch_timestamp
        }
    },
    {
        "$sort": {"_id": 1}  # Optional: Sorting by batch_timestamp if needed
    }
]

results = ProfileModel.objects.aggregate(*pipeline)

# Processing the results to use batch_timestamp as key in a dictionary
grouped_profile_ids = {result['_id']: result['profileIds'] for result in results}

In [1]:
locations = {}
for batch_timestamp, profile_ids in grouped_profile_ids.items():
    print(f"Batch Timestamp: {batch_timestamp}, Profile IDs: {profile_ids}")
    # Now, for each profile_id, query documents with the matching profile_id and batch_timestamp
    for profile_id in profile_ids:
        matching_documents = ProfileModel.objects(
            batch_timestamp=batch_timestamp,
            profileId=profile_id
        )
        # You can convert the QuerySet to a list of dictionaries if needed
        profiles = list(matching_documents.as_pymongo())
        new_locations, profile = localizeProfile(profiles)
        if profile:
            try:
                profile.save()  # Attempt to save the new profile
            except NotUniqueError:
                print("A document with the same unique index already exists. Ignoring.")

        locations.update(new_locations)

NameError: name 'grouped_profile_ids' is not defined

In [ ]:
locations.keys()

In [ ]:
visualize(locations[613297361])